In [ ]:
# link drive https://drive.google.com/drive/folders/18-MEX-KK80-NhISMBUj1K-AsJyr-EuX_

In [29]:
import torch
import torch.nn as nn

import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
import pandas as pd
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import pickle


In [30]:
from Model import Encoder, Decoder, AE_model
from Train import AE_train
from LOSS import AE_loss_function
from sklearnEval import evaluate

In [31]:
# load the data
with open('X_Y.pickle', 'rb') as handle:
    b = pickle.load(handle)
input = torch.from_numpy(b['data'].astype('float'))
test = input
input = input.float()

target = b['label']

In [32]:
DEVICE = torch.device("cpu")
batch_size = 1
x_dim  = 500
hidden_dim = 300
latent_dim = 15
lr = 1e-4

### Step 2. Define our model: Variational AutoEncoder (VAE)

In [33]:
encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim, latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim, hidden_dim = hidden_dim, output_dim = x_dim)
model = AE_model(Encoder=encoder, Decoder=decoder).to(DEVICE)

### Step 3. Define Loss function (reprod. loss) and optimizer

In [34]:
loss_function = AE_loss_function


## Step 4: Train VAE **model**

In [35]:
print("Start training VAE...")
epochs = 200 #
AE_train(dataset = input, model = model, epochs = epochs, loss_ = loss_function, DEVICE = DEVICE)
print("Finish!!")


Start training VAE...
	Epoch 1 complete! 	Average Loss:  0.277831107378006
	Epoch 2 complete! 	Average Loss:  0.27065131068229675
	Epoch 3 complete! 	Average Loss:  0.26152628660202026
	Epoch 4 complete! 	Average Loss:  0.2509188950061798
	Epoch 5 complete! 	Average Loss:  0.2394275814294815
	Epoch 6 complete! 	Average Loss:  0.22770805656909943
	Epoch 7 complete! 	Average Loss:  0.2164417803287506
	Epoch 8 complete! 	Average Loss:  0.20618900656700134
	Epoch 9 complete! 	Average Loss:  0.19703780114650726
	Epoch 10 complete! 	Average Loss:  0.18839876353740692
	Epoch 11 complete! 	Average Loss:  0.17947252094745636
	Epoch 12 complete! 	Average Loss:  0.17039361596107483
	Epoch 13 complete! 	Average Loss:  0.16196048259735107
	Epoch 14 complete! 	Average Loss:  0.15476514399051666
	Epoch 15 complete! 	Average Loss:  0.14892631769180298
	Epoch 16 complete! 	Average Loss:  0.14415787160396576
	Epoch 17 complete! 	Average Loss:  0.14009134471416473
	Epoch 18 complete! 	Average Loss:  0.13

In [8]:
#torch.save(model, 'model_2.pt')

In [36]:
model.eval()
zs = []
for i in input:
    _,z= model(i)
    zs.append(z.detach().numpy())


In [37]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=0, n_init="auto").fit(zs)
predict = kmeans.labels_
target = b['label']
evaluate(target = target, predict = predict)

Adjusted mutual info score: 0.7273187591444649
Adjusted rand score: 0.7387883468769058
Completeness score: 0.7175361622829975
Fowlkes mallow score: 0.8307233329399385
Homogeneity: 0.7380174753302193 
Completeness score: 0.7175361622829975
V measure: 0.7276327209962296
Mutual information between two clusterings score: 0.7783253223297782
Normalized Mutual Information between two clusterings score: 0.7276327209962294
Rand score: 0.880926946946175
